[View in Colaboratory](https://colab.research.google.com/github/OpenPIV/openpiv-python-gpu/blob/master/Openpiv_Python_Cython_GPU_demo.ipynb)

In [0]:

!/opt/bin/nvidia-smi
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb 2> /dev/null
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub
!apt-get update
!apt-get install -qq cuda gcc-5 g++-5 -y
!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc
!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++
!/usr/local/cuda/bin/nvcc --version

In [0]:
# put nvcc on PATH for PyCUDA install
import os
work_dir = os.getcwd()
if "/usr/local/cuda/bin" not in os.environ['PATH'].split(':'):
  os.environ['PATH'] += ":/usr/local/cuda/bin"
  
! echo $PATH
! nvcc --version

! pwd
! ls

In [0]:
# install pycuda using pip
! pip install pycuda

In [0]:
# check to make sure PyCUDA installed properly
import pycuda.autoinit

In [0]:
!pip install cython

In [0]:
!# install scikits-cuda from github as the pip version cant find cusolver
!git clone https://github.com/lebedov/scikit-cuda.git


In [0]:
#!ls
os.chdir(work_dir)
os.chdir('scikit-cuda')
%run setup.py install
%run setup.py test

# add scikit-cuda to the PYTHONPATH
import sys
sys.path.append(os.getcwd())
os.chdir(work_dir)

In [0]:
! pip install progressbar

In [0]:
#!git clone https://github.com/CameronDallas5000/openpiv-python.git
!git clone https://github.com/OpenPIV/openpiv-python-gpu.git
os.chdir('openpiv-python-gpu')
sys.path.append(os.getcwd())
!python setup.py build_ext --inplace
!python setup.py install

In [0]:
import openpiv

In [0]:
from openpiv import tools, process, validation, filters, scaling, pyprocess
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [0]:
!wget https://raw.githubusercontent.com/CameronDallas5000/openpiv-python/master/openpiv/examples/tutorial-part1/exp1_001_a.bmp
!wget https://raw.githubusercontent.com/CameronDallas5000/openpiv-python/master/openpiv/examples/tutorial-part1/exp1_001_b.bmp
!ls

In [0]:
# from google.colab import files
# files.upload() # choose exp1_001_b.bmp and _c.bmp

In [0]:
frame_a  = tools.imread( 'exp1_001_a.bmp' )
frame_b  = tools.imread( 'exp1_001_b.bmp' )

In [0]:
fig,ax = plt.subplots(1,2,figsize=(10,8))
ax[0].imshow(frame_a,cmap=plt.cm.gray)
ax[1].imshow(frame_b,cmap=plt.cm.gray)

# OpenPIV Python version

In [0]:
winsize = 32 # pixels
searchsize = 64  # pixels, search in image B
overlap = 12 # pixels
dt = 0.02 # sec

In [0]:
u, v, sig2noise = pyprocess.extended_search_area_piv( frame_a.astype(np.int32), frame_b.astype(np.int32), 
                                                     window_size=winsize, overlap=overlap, dt=dt, 
                                                     search_area_size=searchsize, sig2noise_method='peak2peak' )

In [0]:
x, y = pyprocess.get_coordinates( image_size=frame_a.shape, window_size=searchsize, overlap=overlap )

In [0]:
tools.save(x, y, u, v, np.zeros_like(u), 'exp1_001.txt' ) # no masking, all values are valid

# OpenPIV Python-GPU version

In [0]:
from progressbar import *


In [0]:
import openpiv.gpu_process
reload(openpiv.gpu_process)

In [0]:
# gpu code parametes
min_window_size = 32
overlap_ratio = 0.5
coarse_factor = 1
nb_iter_max = 2

In [0]:
# First time is slow as the GPU modules need to compile. Once they are compiled, they stay compiledEvery time you run this after the first time it will be fast.x, y, u, v, mask = openpiv.gpu_process.WiDIM( frame_a.astype(np.int32), frame_b.astype(np.int32), np.ones(frame_a.shape, dtype=np.int32),
                                                     min_window_size, 
                                                     overlap_ratio,
                                                     coarse_factor,
                                                     dt,
                                                     nb_iter_max = nb_iter_max)

In [0]:
tools.save(x, y, u, v, np.zeros_like(u), 'exp1_001_gpu.txt' )

# OpenPIV Python-CPU (Cython) version

In [0]:
x, y, u, v, mask = openpiv.process.WiDIM( frame_a.astype(np.int32), frame_b.astype(np.int32), np.ones(frame_a.shape, dtype=np.int32),
                                                     min_window_size, 
                                                     overlap_ratio,
                                                     coarse_factor,
                                                     dt,
                                                     nb_iter_max = nb_iter_max)

In [0]:
tools.save(x, y, u, v, np.zeros_like(u), 'exp1_001_cpu.txt' )

In [0]:
tools.display_vector_field('exp1_001.txt', scale=5000, width=0.0025)
tools.display_vector_field('exp1_001_cpu.txt', scale=5000, width=0.0025)
tools.display_vector_field('exp1_001_gpu.txt', scale=5000, width=0.0025)